In [57]:
import polars as pl
from perspective.widget import PerspectiveWidget

In [58]:
df = pl.read_parquet("stock_trades.parquet")
df = df.sort("交易日期", "交易时间", "证券代码")

In [59]:
df = df.with_columns(
    pl.col("交易时间").cast(pl.String),
    手续费率=pl.col("手续费") / pl.col("成交金额"),
    印花税率=pl.col("印花税") / pl.col("成交金额"),
    过户费率=pl.col("过户费") / pl.col("成交金额"),
)

In [60]:
df = df.with_row_index("序号", 1)

In [61]:
df

序号,券商,交易日期,交易时间,证券代码,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率
u32,str,date,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""湘财""",2022-07-11,"""09:33:37""","""000900""","""现代投资""","""买入""",4.05,34400.0,139320.0,22.29,0.0,1.39,0.0,-139342.29,0.00016,0.0,0.00001
2,"""湘财""",2022-07-11,"""09:34:24""","""601077""","""渝农商行""","""买入""",3.65,38300.0,139795.0,22.37,0.0,1.38,0.0,-139818.75,0.00016,0.0,0.00001
3,"""湘财""",2022-07-11,"""09:36:30""","""600894""","""广日股份""","""买入""",6.54,21400.0,139956.0,22.39,0.0,1.41,0.0,-139979.8,0.00016,0.0,0.00001
4,"""湘财""",2022-07-11,"""09:37:25""","""601992""","""金隅集团""","""买入""",2.59,54000.0,139860.0,22.38,0.0,1.42,0.0,-139883.8,0.00016,0.0,0.00001
5,"""湘财""",2022-07-11,"""09:38:16""","""002462""","""嘉事堂""","""买入""",13.51,10400.0,140504.0,22.48,0.0,1.41,0.0,-140526.48,0.00016,0.0,0.00001
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
359,"""海通两融""",2023-10-31,"""09:31:53""","""002956""","""西麦食品""","""卖出""",14.13,5000.0,70650.0,6.74,35.35,0.0,0.0,70607.91,0.000095,0.0005,0.0
360,"""海通两融""",2023-10-31,"""09:39:57""","""603214""","""爱婴室""","""买入""",15.84,3100.0,49104.0,5.0,0.0,0.51,0.0,-49109.51,0.000102,0.0,0.00001
361,"""海通两融""",2023-10-31,"""09:40:55""","""300132""","""青松股份""","""买入""",5.21,9600.0,50016.0,5.0,0.0,0.0,0.0,-50021.0,0.0001,0.0,0.0


In [62]:
PerspectiveWidget(df)

PerspectiveWidget(binding_mode='server', columns=['序号', '券商', '交易日期', '交易时间', '证券代码', '证券名称', '买卖标志', '成交价格', …

In [62]:
PerspectiveWidget(df)

PerspectiveWidget(binding_mode='server', columns=['序号', '券商', '交易日期', '交易时间', '证券代码', '证券名称', '买卖标志', '成交价格', …

In [76]:
d1 = df.join(
    df.group_by("证券代码", "证券名称")
    .agg(
        结余数量=pl.when(pl.col("买卖标志") == "卖出")
        .then(-pl.col("成交数量"))
        .when(pl.col("买卖标志") == "买入")
        .then(pl.col("成交数量"))
        .sum()
    )
    .filter(pl.col("结余数量") < 0),
    on="证券代码",
    how="anti",
)

In [83]:
start_date = df["交易日期"].min()

In [84]:
end_date = df["交易日期"].max()

In [87]:
k1 = pl.select(日期=pl.date_range(start_date, end_date))

In [92]:
k2 = df["证券代码"].unique().sort().to_frame()
k2

证券代码
str
"""000096"""
"""000532"""
"""000559"""
"""000599"""
"""000655"""
…
"""688299"""
"""688321"""
"""688360"""


In [104]:
k = k1.join(k2, how="cross")

In [116]:
d2 = (
    # 使用left join合并两个DataFrame，通过日期和证券代码匹配
    k.join(
        d1, left_on=["日期", "证券代码"], right_on=["交易日期", "证券代码"], how="left"
    )
    # 按日期和证券代码排序
    .sort(["日期", "证券代码"])
    # 添加一个新列"结余数量"，计算每个证券的累计持仓
    .with_columns(
        结余数量=pl.when(pl.col("买卖标志") == "买入")
        .then(pl.col("成交数量"))
        .when(pl.col("买卖标志") == "卖出")
        .then(-pl.col("成交数量"))
        .otherwise(0)
        .cum_sum()
        .over("证券代码")
    )
    # 筛选出结余数量大于0的记录
).filter(pl.col("结余数量") > 0)
d2

日期,证券代码,序号,券商,交易时间,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率,结余数量
date,str,u32,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-07-11,"""000900""",1,"""湘财""","""09:33:37""","""现代投资""","""买入""",4.05,34400.0,139320.0,22.29,0.0,1.39,0.0,-139342.29,0.00016,0.0,0.00001,34400.0
2022-07-11,"""002462""",5,"""湘财""","""09:38:16""","""嘉事堂""","""买入""",13.51,10400.0,140504.0,22.48,0.0,1.41,0.0,-140526.48,0.00016,0.0,0.00001,10400.0
2022-07-11,"""600894""",3,"""湘财""","""09:36:30""","""广日股份""","""买入""",6.54,21400.0,139956.0,22.39,0.0,1.41,0.0,-139979.8,0.00016,0.0,0.00001,21400.0
2022-07-11,"""601077""",2,"""湘财""","""09:34:24""","""渝农商行""","""买入""",3.65,38300.0,139795.0,22.37,0.0,1.38,0.0,-139818.75,0.00016,0.0,0.00001,38300.0
2022-07-11,"""601992""",4,"""湘财""","""09:37:25""","""金隅集团""","""买入""",2.59,54000.0,139860.0,22.38,0.0,1.42,0.0,-139883.8,0.00016,0.0,0.00001,54000.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-10-31,"""300132""",361,"""海通两融""","""09:40:55""","""青松股份""","""买入""",5.21,9600.0,50016.0,5.0,0.0,0.0,0.0,-50021.0,0.0001,0.0,0.0,9600.0
2023-10-31,"""300215""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,20000.0
2023-10-31,"""600525""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8500.0


In [117]:
PerspectiveWidget(d2)

PerspectiveWidget(binding_mode='server', columns=['日期', '证券代码', '序号', '券商', '交易时间', '证券名称', '买卖标志', '成交价格', '成…

In [119]:
import tushare as ts

pro = ts.pro_api()

In [122]:
f"{start_date:%Y%m%d}"

'20220711'

In [124]:
format(end_date, "%Y%m%d")

'20231031'

In [120]:
hq = pro.daily(
    ts_code="002462.SZ",
    start_date=format(start_date, "%Y%m%d"),
    end_date=format(end_date, "%Y%m%d"),
)

In [125]:
hq = pl.from_pandas(hq)
hq

ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""002462.SZ""","""20231031""",14.75,14.9,14.59,14.7,14.75,-0.05,-0.339,65859.96,96984.271
"""002462.SZ""","""20231030""",13.88,14.89,13.88,14.75,13.93,0.82,5.8866,123932.16,180119.372
"""002462.SZ""","""20231027""",13.7,13.98,13.51,13.93,13.64,0.29,2.1261,35782.0,49386.168
"""002462.SZ""","""20231026""",13.49,13.68,13.4,13.64,13.62,0.02,0.1468,19215.0,26005.866
"""002462.SZ""","""20231025""",13.65,13.77,13.58,13.62,13.67,-0.05,-0.3658,18484.0,25274.163
…,…,…,…,…,…,…,…,…,…,…
"""002462.SZ""","""20220715""",13.61,13.66,13.12,13.13,13.59,-0.46,-3.3848,32967.65,44114.064
"""002462.SZ""","""20220714""",13.54,13.75,13.5,13.59,13.54,0.05,0.3693,21967.0,29851.164
"""002462.SZ""","""20220713""",13.55,13.63,13.39,13.54,13.61,-0.07,-0.5143,22793.0,30714.624


In [156]:
ts_codes = (
    d1.select(
        证券代码=(
            # 检查证券代码的首字符
            pl.when(pl.col("证券代码").str.head(1).is_in(["0", "3"]))
            # 深交所代码（0开头和3开头）添加.SZ后缀
            .then(pl.format("{}.SZ", pl.col("证券代码")))
            # 上交所代码（6开头）添加.Sh后缀
            .when(pl.col("证券代码").str.head(1) == "6")
            .then(pl.format("{}.Sh", pl.col("证券代码")))
            # 对于不符合条件的代码（如首字符不是0/3/6），返回null
        ),
    )
    .to_series()
    .unique()
    .sort()
    .to_list()
)

In [157]:
from tqdm.notebook import tqdm

In [158]:
hq = [
    pl.from_pandas(
        pro.daily(
            ts_code=ts_code,
            start_date=format(start_date, "%Y%m%d"),
            end_date=format(end_date, "%Y%m%d"),
        )
    )
    for ts_code in tqdm(ts_codes)
]

  0%|          | 0/149 [00:00<?, ?it/s]

In [159]:
hq = pl.concat(hq)

In [160]:
!pwd


/c/Users/13212/repo/week08


In [161]:
hq.write_parquet("daily.parquet")

In [162]:
hq = pl.read_parquet("daily.parquet")

In [166]:
hq = hq.with_columns(
    pl.col("ts_code").str.head(6),
    pl.col("trade_date").str.to_date("%Y%m%d"),
)

In [169]:
d1.join(
    hq, left_on=["交易日期", "证券代码"], right_on=["trade_date", "ts_code"], how="left"
).filter(
    ~pl.col("成交价格").is_between(pl.col("low"), pl.col("high")),
)

序号,券商,交易日期,交易时间,证券代码,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率,open,high,low,close,pre_close,change,pct_chg,vol,amount
u32,str,date,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64


In [170]:
d3 = (
    # 使用left join合并两个DataFrame，通过日期和证券代码匹配
    k.join(
        d1, left_on=["日期", "证券代码"], right_on=["交易日期", "证券代码"], how="left"
    )
    # 按日期和证券代码排序
    .sort(["日期", "证券代码"])
    # 添加一个新列"结余数量"，计算每个证券的累计持仓
    .with_columns(
        结余数量=pl.when(pl.col("买卖标志") == "买入")
        .then(pl.col("成交数量"))
        .when(pl.col("买卖标志") == "卖出")
        .then(-pl.col("成交数量"))
        .otherwise(0)
        .cum_sum()
        .over("证券代码")
    )
    # 筛选出结余数量大于0的记录
)

In [171]:
d3

日期,证券代码,序号,券商,交易时间,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率,结余数量
date,str,u32,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-07-11,"""000096""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
2022-07-11,"""000532""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
2022-07-11,"""000559""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
2022-07-11,"""000599""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
2022-07-11,"""000655""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-10-31,"""688299""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
2023-10-31,"""688321""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0
2023-10-31,"""688360""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0


In [198]:
d4 = (
    d3.join(
        hq,
        left_on=["日期", "证券代码"],
        right_on=["trade_date", "ts_code"],
        how="left",
    )
    .sort("证券代码", "日期")
    .with_columns(close=pl.col("close").fill_null(strategy="forward").over("证券代码"))
    .with_columns(持股市值=pl.col("结余数量") * pl.col("close"))
    .group_by("日期")
    .agg(
        pl.col("持股市值").sum().alias("持股市值总和"),
        pl.col("发生金额").sum().alias("发生金额总和"),
    )
    .sort("日期")
    .with_columns(
        转账金额=pl.when(pl.int_range(0, pl.len()) == 0).then(1000000).otherwise(0)
    )
    # 修改这里，使用聚合后的列名
    .with_columns(
        现金余额=(pl.col("转账金额") + pl.col("发生金额总和")).cum_sum(),
    )
)
d4

日期,持股市值总和,发生金额总和,转账金额,现金余额
date,f64,f64,i32,f64
2022-07-11,703040.0,-699551.12,1000000,300448.88
2022-07-12,707714.0,0.0,0,300448.88
2022-07-13,713855.0,0.0,0,300448.88
2022-07-14,710079.0,0.0,0,300448.88
2022-07-15,692377.0,0.0,0,300448.88
…,…,…,…,…
2023-10-27,571195.0,0.0,0,510845.96
2023-10-28,571195.0,0.0,0,510845.96
2023-10-29,571195.0,0.0,0,510845.96


In [199]:
PerspectiveWidget(d4)

PerspectiveWidget(binding_mode='server', columns=['日期', '持股市值总和', '发生金额总和', '转账金额', '现金余额'], table_name='0.413…

In [187]:
d1

序号,券商,交易日期,交易时间,证券代码,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率
u32,str,date,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""湘财""",2022-07-11,"""09:33:37""","""000900""","""现代投资""","""买入""",4.05,34400.0,139320.0,22.29,0.0,1.39,0.0,-139342.29,0.00016,0.0,0.00001
2,"""湘财""",2022-07-11,"""09:34:24""","""601077""","""渝农商行""","""买入""",3.65,38300.0,139795.0,22.37,0.0,1.38,0.0,-139818.75,0.00016,0.0,0.00001
3,"""湘财""",2022-07-11,"""09:36:30""","""600894""","""广日股份""","""买入""",6.54,21400.0,139956.0,22.39,0.0,1.41,0.0,-139979.8,0.00016,0.0,0.00001
4,"""湘财""",2022-07-11,"""09:37:25""","""601992""","""金隅集团""","""买入""",2.59,54000.0,139860.0,22.38,0.0,1.42,0.0,-139883.8,0.00016,0.0,0.00001
5,"""湘财""",2022-07-11,"""09:38:16""","""002462""","""嘉事堂""","""买入""",13.51,10400.0,140504.0,22.48,0.0,1.41,0.0,-140526.48,0.00016,0.0,0.00001
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
359,"""海通两融""",2023-10-31,"""09:31:53""","""002956""","""西麦食品""","""卖出""",14.13,5000.0,70650.0,6.74,35.35,0.0,0.0,70607.91,0.000095,0.0005,0.0
360,"""海通两融""",2023-10-31,"""09:39:57""","""603214""","""爱婴室""","""买入""",15.84,3100.0,49104.0,5.0,0.0,0.51,0.0,-49109.51,0.000102,0.0,0.00001
361,"""海通两融""",2023-10-31,"""09:40:55""","""300132""","""青松股份""","""买入""",5.21,9600.0,50016.0,5.0,0.0,0.0,0.0,-50021.0,0.0001,0.0,0.0


In [206]:
ihq = pro.index_daily(
    ts_code="000300.SH",
    start_date=format(start_date, "%Y%M%d"),
    end_date=format(end_date, "%Y%M%d"),
    fildes="ts_code,trade_date,pct_chg",
)

In [207]:
pl.from_pandas(ihq).write_parquet("index_daily.parquet")

In [212]:
ihq = pl.read_parquet("index_daily.parquet")
ihq = (
    ihq.with_columns(
        pl.col("pct_chg") / 100 + 1,
    )
    .sort("trade_date")
    .with_columns(
        car=pl.col("pct_chg").cum_prod(),
    )
    .with_columns(沪深300=pl.col("car") * 1000000)
)
ihq

ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount,car,沪深300
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""000300.SH""","""20220104""",4917.7653,4957.9835,4961.4527,4874.5337,4940.3733,-22.608,0.995424,1.51534776e8,3.3652e8,0.995424,995424.0
"""000300.SH""","""20220105""",4868.1202,4907.9347,4916.2822,4851.9835,4917.7653,-49.6451,0.989905,1.788161e8,3.6394e8,0.985375,985375.19472
"""000300.SH""","""20220106""",4818.2318,4842.156,4857.5596,4786.433,4868.1202,-49.8884,0.989752,1.57665826e8,3.2175e8,0.975277,975277.069725
"""000300.SH""","""20220107""",4822.3689,4824.3181,4856.6522,4818.1931,4818.2318,4.1371,1.000859,1.87139413e8,3.3222e8,0.976115,976114.832727
"""000300.SH""","""20220110""",4844.0451,4812.23,4844.3928,4780.8191,4822.3689,21.6762,1.004495,1.56211696e8,3.0503e8,0.980502,980502.468901
…,…,…,…,…,…,…,…,…,…,…,…,…
"""000300.SH""","""20221226""",3843.4887,3830.5682,3856.8066,3827.5186,3828.2188,15.2699,1.003989,7.3005712e7,1.4869e8,0.777973,777973.402172
"""000300.SH""","""20221227""",3887.854,3860.9054,3891.8497,3856.58,3843.4887,44.3653,1.011543,7.751535e7,1.5203e8,0.786954,786953.549153
"""000300.SH""","""20221228""",3871.2644,3874.9132,3884.7205,3854.9406,3887.854,-16.5896,0.995733,8.4424656e7,1.6067e8,0.783596,783595.618359


In [214]:
PerspectiveWidget(ihq)

PerspectiveWidget(binding_mode='server', columns=['ts_code', 'trade_date', 'close', 'open', 'high', 'low', 'pr…